In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import b3d
import sys
sys.path.append(b3d.get_root_path())

from tests.sama4d.video_to_tracks.keypoint_tracking_task import KeypointTrackingTask
from tests.sama4d.data_curation import get_cheezitbox_scene_loader
import rerun as rr

In [86]:
scene_spec = get_cheezitbox_scene_loader()
task = KeypointTrackingTask(scene_spec["feature_track_data_loader"], scene_name=scene_spec["scene_name"], n_frames=10)

In [87]:
rr.init(f'task--{task.name}-12')
rr.connect("127.0.0.1:8812")
task.visualize_task()

In [88]:
from tests.sama4d.video_to_tracks.patch_tracking_solver import AdamPatchTracker
task_spec = task.get_task_specification()
solver = AdamPatchTracker()
solution = solver.solve(task_spec)

In [89]:
metrics = task.score(solution)

In [90]:
task.visualize_solution(solution, metrics)

In [91]:
import rerun.blueprint as rrb
bp = rrb.Blueprint(
    rrb.Horizontal(
        rrb.Spatial3DView(),
        rrb.Vertical(
            rrb.Spatial2DView(contents=["groundtruth_solution/keypoints_2d", "solution/keypoints_2D"]),
            rrb.Spatial2DView()
        )
    )
)
rr.send_blueprint(bp)

In [92]:
solver.visualize_solver_state(task_spec)